<a href="https://colab.research.google.com/github/amalsalilan/B3-Developing-Named-Entity-Recognition-NER-Models-for-Financial-Data-Extraction-/blob/Sameer/NER_SPRINGBOARD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy -q


In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.6.105+-x86_64-with-glibc2.35
Python version   3.12.12                       
Pipelines        en_core_web_sm (3.8.0)        



In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp=spacy.blank("en") #load a new spacy model
db=DocBin() #Create a DocBin object

In [ ]:
import json
f=open('annotations.json')
TRAIN_DATA=json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./annotations.spacy")

100%|██████████| 2/2 [00:00<00:00, 122.07it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:

! python -m spacy train config.cfg --output ./ --paths.train ./annotations.spacy --paths.dev ./annotations.spacy


ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    181.50    2.44    2.17    2.78    0.02
100     200       4491.13   7026.36   56.00  100.00   38.89    0.56
200     400        120.26   2845.97   56.00  100.00   38.89    0.56
300     600         89.85   2707.12   59.26   88.89   44.44    0.59
400     800         81.53   2608.34   56.00  100.00   38.89    0.56
500    1000         79.78   2624.71   76.09   62.50   97.22    0.76
600    1200         66.67   2558.00   56.00  100.00   38.89    0.56
700    1400         69.09   2715.36   56.00  100.00   38.89    0.56
800    1600         67.93   2668.63 

In [ ]:

nlp_ner = spacy.load("/content/model-best")

In [ ]:
doc=nlp_ner(''' The cryptocurrency market saw a strong rebound today as Bitcoin climbed past $79,000, marking a 4.6% increase in 24 hours amid renewed optimism over ETF inflows and positive macroeconomic data. Ethereum followed the trend, rising 3.2% to trade near $2,950, while Solana gained 6% as network activity spiked due to new DeFi protocols launching this week.

Analysts attribute the bullish sentiment to improving investor confidence and decreasing U.S. Treasury yields, which often drive capital toward risk assets. Meanwhile, the total crypto market capitalization has crossed $3.1 trillion for the first time since mid-2022.
In regulatory developments, the European Union announced updated guidelines on stablecoins, emphasizing transparency and reserve backing. Experts believe this could pave the way for institutional adoption across Europe.
On the innovation front, Ripple Labs unveiled a new cross-border payment feature integrated with central bank digital currency (CBDC) networks, signaling a step forward for blockchain-based financial infrastructure.
Despite the optimism, market watchers warn traders to remain cautious, citing volatility and potential corrections ahead as Bitcoin approaches major resistance near $80,000''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)